In [39]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.stats import norm

In [5]:
employees_df = pd.read_csv("employees.csv", sep = ';')
courses_passing_df = pd.read_csv("courses_passing.csv", sep = ';')
course_employee_sms_df = pd.read_csv("course_employee_sms.csv", sep = ';')
communications_df = pd.read_csv("communications.csv", sep = ';')
ab_test_df = pd.read_csv("ab_test.csv", sep = ',')

In [7]:
ab_test_df.head(10)

,employee_id,course_id,after,before,not in range
0,00411460-f7c9-2d21-24a6-7ea0f4cb5f85,7,51.180723,44.162162,57.708428
1,00411460-f7c9-2d21-24a6-7ea0f4cb5f85,8,58.879121,59.816176,56.718228
2,00411460-f7c9-2d21-24a6-7ea0f4cb5f85,20,61.271429,54.013453,57.233972
3,00411460-f7c9-2d21-24a6-7ea0f4cb5f85,29,59.250000,55.598958,57.155440
4,00411460-f7c9-2d21-24a6-7ea0f4cb5f85,50,62.396396,58.080808,56.226648
5,006f52e9-102a-8d3b-e2fe-5614f42ba989,10,58.219780,56.754717,62.112515
6,006f52e9-102a-8d3b-e2fe-5614f42ba989,14,65.832669,66.713415,60.066751
7,006f52e9-102a-8d3b-e2fe-5614f42ba989,17,56.022857,57.300000,62.247914
8,006f52e9-102a-8d3b-e2fe-5614f42ba989,19,57.697095,56.022857,62.471330
9,006f52e9-102a-8d3b-e2fe-5614f42ba989,21,61.704000,61.592760,61.270972


In [15]:
emploee_after = ab_test_df.groupby('employee_id').agg({'after':['mean','std']})
emploee_after

after          
                                           mean       std
employee_id                                              
00411460-f7c9-2d21-24a6-7ea0f4cb5f85  58.595534  4.390192
006f52e9-102a-8d3b-e2fe-5614f42ba989  61.528716  3.850593
00a03ec6-533c-a7f5-c644-d198d815329c  61.133027  3.873031
00ac8ed3-b432-7bdd-4ebb-ebcb2ba10a00  57.386557  1.409761
00e26af6-ac3b-1c1c-49d7-c3d79c60d000  56.491046  2.098407
...                                         ...       ...
ff1418e8-cc99-3fe8-abcf-e3ce2003e5c5  59.075581  2.511347
ff49cc40-a889-0e6a-60f4-0ff3026d2730  56.188235       NaN
fface838-5abb-f94b-4593-a0ed53a0c70f  57.994813  3.180945
ffedf5be-3a86-e2ee-281d-54cdc97bc1cf  57.398029  1.871024
ffeed84c-7cb1-ae7b-f4ec-4bd78275bb98  61.811531  4.939949

[1677 rows x 2 columns]

In [13]:
emploee_after.agg({'after':['mean', 'std']})

,after
mean,59.089710
std,2.336219


In [16]:
ab_test_df.agg({'after':['mean', 'std']})

,after
mean,60.011519
std,3.413131


In [17]:
employees_df.head()

,employee_id,sex,region,age,head_employee_id,exp_days,edu_degree,department_id,work_online_flg
0,0f840be9-b8db-4d3f-bd5b-a2ce59211f55,M,4,36,c81e728d-9d4c-2f63-6f06-7f89cc14862c,734,0,2,1
1,ac1dd209-cbcc-5e5d-1c6e-28598e8cbbe8,M,4,32,93db85ed-909c-1383-8ff9-5ccfa94cebd9,654,0,2,1
2,a8c88a00-55f6-36e4-a163-a5e3d16adab7,F,5,34,9a115815-4dfa-42ca-ddbd-0694a4e9bdc8,276,1,2,1
3,c91591a8-d461-c286-9b9f-535ded3e213e,F,4,33,9f61408e-3afb-633e-50cd-f1b20de6f466,50,2,0,1
4,bc731692-9fe1-545b-f0b9-8d114ee3ecb8,M,0,33,a97da629-b098-b75c-294d-ffdc3e463904,274,1,2,1


In [18]:
courses_passing_df

,course_id,employee_id,pass_frac,start_dt,last_activity_dt,end_dt,educ_duration_days
0,24,53ed35c7-4a2e-c275-b837-374f04396c03,1.00,2023-01-30 00:00:00,2023-02-05 00:00:00,2023-02-05 00:00:00,6.0
1,83,202cb962-ac59-075b-964b-07152d234b70,1.00,2023-08-29 00:00:00,2023-09-10 00:00:00,2023-09-10 00:00:00,12.0
2,4,05f971b5-ec19-6b8c-65b7-5d2ef8267331,1.00,2023-07-29 00:00:00,2023-08-09 00:00:00,2023-08-09 00:00:00,11.0
3,79,f4a331b7-a22d-1b23-7565-d8813a34d8ac,1.00,2023-05-30 00:00:00,2023-06-09 00:00:00,2023-06-09 00:00:00,10.0
4,6,aba3b6fd-5d18-6d28-e06f-f97135cade7f,1.00,2023-07-12 00:00:00,2023-07-22 00:00:00,2023-07-22 00:00:00,10.0
...,...,...,...,...,...,...,...
10945,49,3147da8a-b4a0-437c-15ef-51a5cc7f2dc4,1.00,2023-05-11 00:00:00,2023-05-18 00:00:00,2023-05-18 00:00:00,7.0
10946,16,00e26af6-ac3b-1c1c-49d7-c3d79c60d000,1.00,2023-05-08 00:00:00,2023-05-20 00:00:00,2023-05-20 00:00:00,12.0
10947,40,e5b294b7-0c96-47dc-f804-d7baa1903918,1.00,2023-03-30 00:00:00,2023-04-13 00:00:00,2023-04-13 00:00:00,14.0
10948,24,fb89705a-e6d7-43bf-1e84-8c206e16a1d7,0.05,2023-05-05 00:00:00,2023-05-08 00:00:00,NaN,NaN


In [19]:
courses_passing_df["start_dt"] = pd.to_datetime(courses_passing_df["start_dt"])
courses_passing_df["last_activity_dt"] = pd.to_datetime(courses_passing_df["last_activity_dt"])
courses_passing_df["end_dt"] = pd.to_datetime(courses_passing_df["end_dt"])

In [53]:
employee_firstday = courses_passing_df.groupby('employee_id').agg({'start_dt':'min', 'employee_id':'min'})
employee_firstday

,start_dt,employee_id
employee_id,,
00411460-f7c9-2d21-24a6-7ea0f4cb5f85,2023-01-11,00411460-f7c9-2d21-24a6-7ea0f4cb5f85
006f52e9-102a-8d3b-e2fe-5614f42ba989,2023-03-07,006f52e9-102a-8d3b-e2fe-5614f42ba989
008bd5ad-93b7-54d5-0033-8c253d9c1770,2023-03-13,008bd5ad-93b7-54d5-0033-8c253d9c1770
00a03ec6-533c-a7f5-c644-d198d815329c,2023-01-12,00a03ec6-533c-a7f5-c644-d198d815329c
00ac8ed3-b432-7bdd-4ebb-ebcb2ba10a00,2023-02-11,00ac8ed3-b432-7bdd-4ebb-ebcb2ba10a00
...,...,...
ff1418e8-cc99-3fe8-abcf-e3ce2003e5c5,2023-03-04,ff1418e8-cc99-3fe8-abcf-e3ce2003e5c5
ff49cc40-a889-0e6a-60f4-0ff3026d2730,2023-03-27,ff49cc40-a889-0e6a-60f4-0ff3026d2730
fface838-5abb-f94b-4593-a0ed53a0c70f,2023-02-13,fface838-5abb-f94b-4593-a0ed53a0c70f


In [61]:
dist_employee_firstday = employee_firstday.groupby('start_dt').agg({'employee_id': 'count'}).query('start_dt >= "2023-03-01" and start_dt < "2023-05-01"')
dist_employee_firstday.agg({'employee_id': ['mean', 'sum']})

,employee_id
mean,11.557377
sum,705.000000


In [62]:
dist_employee_firstday

,employee_id
start_dt,
2023-03-01,17
2023-03-02,18
2023-03-03,12
2023-03-04,18
2023-03-05,16
...,...
2023-04-26,7
2023-04-27,4
2023-04-28,9


In [71]:
px.bar(x = list(dist_employee_firstday.index), y = dist_employee_firstday['employee_id'])

In [40]:
def estimate_mde(mean, sigma, n, alpha = 0.05, power = 0.8):
    coef = norm.ppf(q = 1 - (alpha / 2)) + norm.ppf(q = power)
    return coef * (np.sqrt(2 * sigma**2 / n)) / mean

In [70]:
estimate_mde(60.011519, 3.413131, 30 * 6)

0.016795808728472007

In [69]:
estimate_mde(60.011519, 3.413131, 45 * 6)

0.013713720400713452

In [68]:
estimate_mde(60.011519, 3.413131, 60 * 6)

0.01187643024741476